In [1]:
import tensorflow as tf

# SimpleRNN - intro

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU

In [3]:
model = Sequential()

model.add( GRU(200, return_sequences=True, input_shape=(15, 100)) )
model.add( GRU(500))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 15, 200)           181200    
_________________________________________________________________
gru_1 (GRU)                  (None, 500)               1053000   
Total params: 1,234,200
Trainable params: 1,234,200
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.input

<tf.Tensor 'gru_input:0' shape=(None, 15, 100) dtype=float32>

In [5]:
model.output

<tf.Tensor 'gru_1/Identity:0' shape=(None, 500) dtype=float32>

# Dataset

In [6]:
import numpy as np
from tensorflow.keras.datasets import reuters
max_words  = 400
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words, test_split=0.2)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Loading data...
8982 train sequences
2246 test sequences
46 classes


In [7]:
len(x_train[0]), len(x_train[1]), len(x_train[14])

(87, 56, 527)

In [8]:
x_train[1]

[1,
 2,
 2,
 2,
 2,
 56,
 2,
 2,
 9,
 56,
 2,
 2,
 81,
 5,
 2,
 57,
 366,
 2,
 132,
 20,
 2,
 7,
 2,
 49,
 2,
 2,
 2,
 2,
 2,
 2,
 8,
 7,
 10,
 241,
 16,
 2,
 129,
 231,
 2,
 5,
 4,
 2,
 2,
 2,
 2,
 2,
 7,
 48,
 34,
 191,
 44,
 35,
 2,
 2,
 17,
 12]

In [9]:
# Sprawdzamy, czy gdziekolwiek użyto zerowego kodu słowa
np.sum( np.array( [np.sum( np.array( x_train[i] ) == 0 ) for i in range(len(x_train))] ) )

0

In [10]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (8982, 46)
y_test shape: (2246, 46)


## Sequence padding

In [11]:
from tensorflow.keras.preprocessing import sequence

maxlen = 200

print('Pad sequences (samples x time)')
x_train_padded = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test_padded  = sequence.pad_sequences(x_test,  maxlen=maxlen)
print('x_train        shape:', x_train.shape)
print('x_test         shape:', x_test.shape)
print('x_train_padded shape:', x_train_padded.shape)
print('x_test_padded  shape:', x_test_padded.shape)

Pad sequences (samples x time)
x_train        shape: (8982,)
x_test         shape: (2246,)
x_train_padded shape: (8982, 200)
x_test_padded  shape: (2246, 200)


In [12]:
x_train_padded[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   2,   2,   8,
        43,  10,   2,   5,  25, 207, 270,   5,   2, 111,  16, 369, 186,
        90,  67,   7,  89,   5,  19, 102,   6,  19, 124,  15,  90,  67,
        84,  22,   2,  26,   7,  48,   4,  49,   8,   2,  39, 209, 154,
         6, 151,   6,  83,  11,  15,  22, 155,  11,  15,   7,  48,   9,
         2,   2,   2,   6, 258,   6, 272,  11,  15,  22, 134,  4

# Embedding

<img src="https://image.slidesharecdn.com/translatefrombadenglishtogoodone-2-160606105036/95/aibigdata-lab-2016-11-638.jpg?cb=1465210454" width="700">

In [13]:
from tensorflow.keras.layers import Embedding, Dropout, Dense

In [14]:
embeddings_dim = 128

model = Sequential()
model.add( Embedding( 400 , embeddings_dim , input_length=200)  )
# model.add( Dropout(0.2) )

model.add( GRU(250, return_sequences=True) )
model.add( GRU(500) )

model.add( Dense(46, activation='softmax') )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          51200     
_________________________________________________________________
gru_2 (GRU)                  (None, 200, 250)          285000    
_________________________________________________________________
gru_3 (GRU)                  (None, 500)               1128000   
_________________________________________________________________
dense (Dense)                (None, 46)                23046     
Total params: 1,487,246
Trainable params: 1,487,246
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.input

<tf.Tensor 'embedding_input:0' shape=(None, 200) dtype=float32>

In [16]:
model.output

<tf.Tensor 'dense/Identity:0' shape=(None, 46) dtype=float32>

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
model.fit(x_train_padded, y_train, batch_size=32, epochs=1, verbose=1, validation_split=0.1 )

Train on 8083 samples, validate on 899 samples
8083/8083 [==============================] - 21s 3ms/sample - loss: 2.0621 - accuracy: 0.4617 - val_loss: 1.8135 - val_accuracy: 0.5184


In [26]:
model.layers[0].weights

[<tf.Variable 'embedding_3/embeddings:0' shape=(400, 128) dtype=float32, numpy=
 array([[ 0.02655471,  0.0424326 ,  0.01030646, ...,  0.01516584,
          0.01014158,  0.05025052],
        [ 0.03735615, -0.00410256, -0.01206659, ...,  0.02475554,
         -0.04505358, -0.02580771],
        [ 0.0288943 , -0.00987562,  0.04722525, ...,  0.0110033 ,
         -0.00849142,  0.03443868],
        ...,
        [ 0.00135914,  0.00666283,  0.00708391, ...,  0.04539788,
          0.00697697, -0.02247263],
        [ 0.01823797, -0.02547565, -0.00559118, ...,  0.02901956,
         -0.01527192, -0.01932083],
        [-0.00689064,  0.02441138, -0.02999184, ...,  0.04719028,
          0.03238411,  0.03331821]], dtype=float32)>]

# Word2Vec

In [19]:
import numpy as np
import gensim

from   gensim.models import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format('/home/rno/data/models/GoogleNews-vectors-negative300.bin.gz', binary=True)

# GloVe

In [21]:
import os
import numpy as np

BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, '/home/datasets/models/glove.6B')
# EMBEDDING_DIM = 100

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [25]:
embeddings_index['mother']

array([ 0.60587 ,  0.027989,  0.018495, -0.018674, -0.39562 ,  1.0309  ,
       -0.35793 ,  0.20527 ,  0.3293  ,  0.035267, -0.38475 ,  0.31452 ,
        0.32538 ,  0.70024 ,  0.13935 , -0.58923 ,  0.36985 , -0.080566,
       -0.59721 ,  1.0215  , -0.55154 ,  0.042073,  0.34687 ,  0.86511 ,
        0.63521 ,  0.52616 , -0.92199 , -1.4634  ,  0.34517 ,  0.58921 ,
        0.12295 ,  0.7323  ,  1.0468  ,  0.065458, -0.27033 , -0.095179,
        0.20613 ,  0.22589 ,  0.90409 , -0.11252 , -0.58059 ,  0.036599,
        0.32003 , -0.53638 ,  0.19297 ,  0.035694, -0.56487 ,  0.1527  ,
        0.70196 , -0.24191 ,  0.10476 , -0.23424 ,  1.212   ,  1.1612  ,
       -0.033677, -1.9996  , -0.79448 , -0.087088,  0.51475 ,  0.44601 ,
        0.638   ,  0.89893 ,  0.17408 , -0.32006 ,  0.41652 ,  0.23289 ,
        0.50642 ,  0.26938 , -0.1453  ,  0.1207  , -0.26246 ,  0.16991 ,
        0.16702 , -0.042041,  0.64841 ,  0.9827  , -0.092602, -0.56797 ,
       -0.63854 , -0.38415 , -0.13816 ,  0.43137 , 

In [27]:
x_train_padded[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   2,   2,   8,
        43,  10,   2,   5,  25, 207, 270,   5,   2, 111,  16, 369, 186,
        90,  67,   7,  89,   5,  19, 102,   6,  19, 124,  15,  90,  67,
        84,  22,   2,  26,   7,  48,   4,  49,   8,   2,  39, 209, 154,
         6, 151,   6,  83,  11,  15,  22, 155,  11,  15,   7,  48,   9,
         2,   2,   2,   6, 258,   6, 272,  11,  15,  22, 134,  4

In [28]:
# 
embedding_matrix = np.random.randn(400, 100)

In [29]:
embeddings_dim = 100

model = Sequential()
model.add( Embedding( 400 , embeddings_dim , input_length=200, weights=[embedding_matrix], trainable=False)  )
# model.add( Dropout(0.2) )

model.add( GRU(250, return_sequences=True) )
model.add( GRU(500) )

model.add( Dense(46, activation='softmax') )
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          40000     
_________________________________________________________________
gru_4 (GRU)                  (None, 200, 250)          264000    
_________________________________________________________________
gru_5 (GRU)                  (None, 500)               1128000   
_________________________________________________________________
dense_1 (Dense)              (None, 46)                23046     
Total params: 1,455,046
Trainable params: 1,415,046
Non-trainable params: 40,000
_________________________________________________________________


In [ ]:
model.layers[0].trainable = False